# 保存与加载

上一章节主要介绍了如何调整超参数，并进行网络模型训练。在训练网络模型的过程中，实际上我们希望保存中间和最后的结果，用于微调（fine-tune）和后续的模型推理与部署，本章节我们将介绍如何保存与加载模型。

In [ ]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.2.14，如需更换mindspore版本，可更改下面mindspore的版本号
!pip uninstall mindspore -y
!pip install -i https://pypi.mirrors.ustc.edu.cn/simple mindspore==2.2.14

In [ ]:
import numpy as np
import mindspore
from mindspore import nn
from mindspore import Tensor

In [ ]:
def network():
    model = nn.SequentialCell(
                nn.Flatten(),
                nn.Dense(28*28, 512),
                nn.ReLU(),
                nn.Dense(512, 512),
                nn.ReLU(),
                nn.Dense(512, 10))
    return model

## 保存和加载模型权重

保存模型使用`save_checkpoint`接口，传入网络和指定的保存路径：

In [ ]:
model = network()
mindspore.save_checkpoint(model, "model.ckpt")

要加载模型权重，需要先创建相同模型的实例，然后使用`load_checkpoint`和`load_param_into_net`方法加载参数。

In [ ]:
model = network()
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)

> `param_not_load`是未被加载的参数列表，为空时代表所有参数均加载成功。

## 保存和加载MindIR

除Checkpoint外，MindSpore提供了云侧（训练）和端侧（推理）统一的[中间表示（Intermediate Representation，IR）](https://www.mindspore.cn/docs/zh-CN/r2.3.0rc2/design/all_scenarios.html#中间表示mindir)。可使用`export`接口直接将模型保存为MindIR。

In [ ]:
model = network()
inputs = Tensor(np.ones([1, 1, 28, 28]).astype(np.float32))
mindspore.export(model, inputs, file_name="model", file_format="MINDIR")

> MindIR同时保存了Checkpoint和模型结构，因此需要定义输入Tensor来获取输入shape。

已有的MindIR模型可以方便地通过`load`接口加载，传入`nn.GraphCell`即可进行推理。

> `nn.GraphCell`仅支持图模式。

In [ ]:
mindspore.set_context(mode=mindspore.GRAPH_MODE)

graph = mindspore.load("model.mindir")
model = nn.GraphCell(graph)
outputs = model(inputs)
print(outputs.shape)